# Create observing files for session starting on 2024-11-23 UTC

In [10]:
from functools import partial
import importlib
from sqlite3 import connect
import sys

import astroplan as ap
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
import numpy as np
import pandas as pd

sys.path.append("../../")
import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.phase as ph
import astropaul.priority as pr

importlib.reload(tlc)
importlib.reload(lbt)
importlib.reload(ph)
importlib.reload(pr)

<module 'astropaul.priority' from 'c:\\Users\\User\\Dropbox\\Astro\\Other\\2024-11-14 LBT Observing\\../..\\astropaul\\priority.py'>

In [94]:
importlib.reload(tlc)
importlib.reload(lbt)

conn = connect("../../django/TargetDB/db.sqlite3")

session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
session.add_day_range("2024-11-23", "2024-11-27 12:00")

already_viewed_2024B = {
    "TIC 89278612",  # 2024-09-20 05:17:18
    "TIC 470710327",  # 2024-09-20 06:46:13
    "TIC 367448265",  # 2024-09-20 08:35:06
    "TIC 260056937",  # 2024-09-20 10:39:05
    "TIC 392229331",  # 2024-09-20 10:53:46
    "TIC 286470992",  # 2024-09-20 11:17:00
    "TIC 470710327",  # 2024-09-22 04:07:37
    "TIC 278352276",  # 2024-10-30 01:34:24
    "TIC 414026507",  # 2024-10-31 02:12:56
    "TIC 427092089",  # 2024-10-31 02:22:29
    "TIC 52856877",  # 2024-11-01 06:02:56
}
already_viewed_2024B.remove("TIC 470710327") # this one is special and deserves additional followup observations

calibration_targets_to_use = {
    "HD 23169",
    "HD 65583",
    "HD 693",
    "HD 112299",
    "HD 102870",
    "HIP 104548",
    "HIP 16518",
    "HIP 38979",
}

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": False,
}

name = "2024-11-23 LBT Observing List"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"] == "QuadEB")),
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Name"].isin(already_viewed_2024B) == False)),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    tlc.add_speckle,
    tlc.add_spectra,
    tlc.add_lists,
    tlc.add_ephemerides,
    partial(tlc.filter_targets, criteria=lambda df: (df["Spectra Count"] > 0)),
    partial(tlc.filter_targets, criteria=lambda df: (df["List LBT 2024B"])),
    partial(tlc.concat_dataframe, other_df=pd.read_csv("Calibration Targets.csv")),
    # partial(tlc.filter_targets, inverse=True, criteria=lambda df: (df["Target Type"].str.contains("Standard")) & (df["Target Name"].isin(calibration_targets_to_use) == False)),
    partial(tlc.add_observability, observing_session=session, calc_max_altitude=True, calc_moon_distance=True),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Every Night"])),
    # partial(
    #     tlc.filter_targets, inverse=True, criteria=lambda df: (df["Target Type"] == "QuadEB") & (df["Observable Max Alt"] < 45)
    # ),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: np.isnan(df["Teff"])),
    partial(lbt.add_pepsi_params, **pepsi_args),
    partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 660),
]
tl = creator.calculate(verbose=False)

tl.target_list["PEPSI notes"] = [type.replace("QuadEB", "Science Target") for type in tl.target_list["Target Type"]]

# remove secondary columns
# for group, (_, cols) in tl.column_groups.items():
#     if group == "Target":
#         continue
#     tl.target_list.drop(columns=cols, inplace=True)

print(tl.summarize())

tl.to_html(f"{name}.html")
# lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list)
print(f"{np.sum(tl.target_list["PEPSI exp_time"])/60:.1f} minutes")
tl.target_list[["Target Name", "RA", "Dec", "Observable Min Moon Dist"]]


2024-11-23 LBT Observing List
23 targets:
      11 QuadEB
       9 RV Standard
       3 Telluric Standard
Columns (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    Speckle Count: (1, 0)
    Spectra Count: (1, 0)
    List : (0, 10)
    Observable: (4, 15)
    PEPSI : (3, 6)
Other tables:
     783 Speckle
     103 Spectra
     673 Ephem
       5 Lunar Phases

81.1 minutes


,Target Name,RA,Dec,Observable Min Moon Dist
4219,TIC 25818450,23:30:58.43,+53:04:08.94,111.374395
4225,TIC 470710327,23:49:18.97,+61:57:46.03,102.216072
4232,TIC 123098844,18:38:17.48,+44:41:54.96,91.923099
4236,TIC 322727163,20:38:51.99,+50:28:00.55,111.954272
4253,TIC 78568780,06:51:23.75,-22:10:01.94,60.794005
4277,TIC 454140642,04:19:05.63,+00:54:00.15,88.181604
4280,TIC 336882813,06:09:33.75,+14:37:44.35,58.744577
4300,TIC 307119043,00:59:18.61,+51:13:17.91,105.866071
4306,TIC 283940788,00:35:24.34,+62:54:05.74,98.773130
4313,TIC 328181241,02:52:36.95,+03:20:52.38,108.538295


In [75]:
importlib.reload(tlc)
importlib.reload(pr)
importlib.reload(ph)

illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.75),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.85),
    ((45, 60), 0.95),
    ((60, 90), 1),
]

phase_defs = [
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.7,
    "B|T": 0.4,
    "R|R": 0.9,
    "R|T": 1,
    "T|T": 0.1,
}

overall_weightings = {"Moon": 1, "Altitude": 1, "Phase": 1, "HQND": 1}

session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
session.add_day_range("2024-11-23", "2024-11-27 12:00")
science_tl = tl.copy()
science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"] == "QuadEB"]
pl = pr.PriorityList(science_tl, session)
pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.calculate_list_priority(pl, "HQND", false_value=0.75)
pr.calculate_phase_priority(pl, phase_defs=phase_defs, phase_categories=phase_scores)
pr.calculate_overall_priority(pl, weightings=overall_weightings)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.1)
pr.aggregate_table_to_html(pl.target_priorities[0], "Target Priorities.html")
print(np.max(pl.target_priorities[0]))
pl.target_priorities[0]

 '2024-11-23T03:00:00.000000000' '2024-11-23T04:00:00.000000000'
 '2024-11-23T05:00:00.000000000' '2024-11-23T06:00:00.000000000'
 '2024-11-23T07:00:00.000000000' '2024-11-23T08:00:00.000000000'
 '2024-11-23T09:00:00.000000000' '2024-11-23T10:00:00.000000000'
 '2024-11-23T11:00:00.000000000' '2024-11-23T12:00:00.000000000'
 '2024-11-23T13:00:00.000000000'], obsgeoloc=[( 4531761.45186297, -2902831.17743518, 3416902.08714099),
 ( 5130134.61505458, -1629477.71730935, 3415410.92324197),
 ( 5377558.88172245,  -244464.18134866, 3414763.28784806),
 ( 5257080.77673768,  1157308.49219577, 3415003.5629756 ),
 ( 4776955.44549111,  2479791.03120173, 3416115.29192937),
 ( 3970081.01245582,  3632367.1127579 , 3418022.30640498),
 ( 2891744.40616787,  4536062.38130766, 3420593.94501478),
 ( 1615833.10639401,  5128955.76076831, 3423654.00570093),
 (  229772.38405363,  5370422.27825974, 3426992.81865124),
 (-1171465.0664738 ,  5243916.68173337, 3430381.61251185),
 (-2491866.64346817,  4758107.11839912, 

0.85


,TIC 283940788,TIC 307119043,TIC 389836747,TIC 328181241,TIC 454140642,TIC 336882813,TIC 78568780,TIC 123098844,TIC 322727163,TIC 25818450,TIC 470710327
2024-11-23 01:00,0.49875,0.35625,0.25500,0.00000,0.000,0.00000,0.00000,0.665,0.52500,0.375,0.28500
2024-11-23 02:00,0.64125,0.49875,0.35625,0.00000,0.000,0.00000,0.00000,0.595,0.52500,0.300,0.28500
2024-11-23 03:00,0.64125,0.52500,0.35625,0.31875,0.000,0.00000,0.00000,0.000,0.35625,0.300,0.30000
2024-11-23 04:00,0.64125,0.52500,0.37500,0.35625,0.000,0.00000,0.00000,0.000,0.25500,0.300,0.35625
2024-11-23 05:00,0.49875,0.52500,0.37500,0.35625,0.595,0.00000,0.00000,0.000,0.00000,0.285,0.35625
2024-11-23 06:00,0.49875,0.52500,0.35625,0.37500,0.665,0.44625,0.00000,0.000,0.00000,0.285,0.35625
2024-11-23 07:00,0.49875,0.49875,0.35625,0.35625,0.665,0.49875,0.00000,0.000,0.00000,0.255,0.44625
2024-11-23 08:00,0.63750,0.44625,0.35625,0.35625,0.665,0.52500,0.00000,0.000,0.00000,0.000,0.44625
2024-11-23 09:00,0.00000,0.44625,0.25500,0.31875,0.665,0.52500,0.00000,0.000,0.00000,0.000,0.00000
2024-11-23 10:00,0.00000,0.00000,0.00000,0.00000,0.850,0.37500,0.31875,0.000,0.00000,0.000,0.00000


In [78]:
foo = pl.target_priorities[0]
bins = [0.00, 0.10, 0.20, 0.5, 1.00]
labels = ["", "*", "* *", "* * *"]
ranking = foo.apply(lambda col: pd.cut(col, bins=bins, labels=labels, ordered=False, include_lowest=True))
ranking

,TIC 283940788,TIC 307119043,TIC 389836747,TIC 328181241,TIC 454140642,TIC 336882813,TIC 78568780,TIC 123098844,TIC 322727163,TIC 25818450,TIC 470710327
2024-11-23 01:00,* *,* *,* *,,,,,* * *,* * *,* *,* *
2024-11-23 02:00,* * *,* *,* *,,,,,* * *,* * *,* *,* *
2024-11-23 03:00,* * *,* * *,* *,* *,,,,,* *,* *,* *
2024-11-23 04:00,* * *,* * *,* *,* *,,,,,* *,* *,* *
2024-11-23 05:00,* *,* * *,* *,* *,* * *,,,,,* *,* *
2024-11-23 06:00,* *,* * *,* *,* *,* * *,* *,,,,* *,* *
2024-11-23 07:00,* *,* *,* *,* *,* * *,* *,,,,* *,* *
2024-11-23 08:00,* * *,* *,* *,* *,* * *,* * *,,,,,* *
2024-11-23 09:00,,* *,* *,* *,* * *,* * *,,,,,
2024-11-23 10:00,,,,,* * *,* *,* *,,,,


In [80]:
tic = "TIC 470710327"
print(pl.target_list.target_list[pl.target_list.target_list["Target Name"] == tic]["Observable Max Alt"])
pl.tables[tic][0]

4225    60.595523
Name: Observable Max Alt, dtype: float64


,Moon Illumination,Moon Illumination Name,Moon Distance,Moon Priority,Altitude Value,Altitude Priority,HQND Priority,Phase System A State,Phase System B State,Phase Priority,Overall Priority
2024-11-23 01:00:00,0.503230,Gray,102.500249,1,54.925431,0.95,0.75,T,B,0.4,0.28500
2024-11-23 02:00:00,0.499182,Gray,102.833918,1,59.072753,0.95,0.75,T,B,0.4,0.28500
2024-11-23 03:00:00,0.495137,Gray,103.153445,1,60.595632,1.00,0.75,T,B,0.4,0.30000
2024-11-23 04:00:00,0.491096,Gray,103.457975,1,59.043388,0.95,0.75,B,B,0.5,0.35625
2024-11-23 05:00:00,0.487059,Gray,103.747568,1,54.874712,0.95,0.75,B,B,0.5,0.35625
2024-11-23 06:00:00,0.483027,Gray,104.023172,1,49.017533,0.95,0.75,B,B,0.5,0.35625
2024-11-23 07:00:00,0.478999,Gray,104.286541,1,42.292539,0.85,0.75,R,B,0.7,0.44625
2024-11-23 08:00:00,0.474976,Gray,104.540108,1,35.277126,0.85,0.75,R,B,0.7,0.44625
2024-11-23 09:00:00,0.470958,Gray,104.786819,1,28.378411,0.00,0.75,R,B,0.7,0.00000
2024-11-23 10:00:00,0.466945,Gray,105.029925,1,21.913386,0.00,0.75,R,B,0.7,0.00000


In [ ]:
target_list = pl.target_list.target_list
this_table_target_list = target_list[target_list["Target Name"].isin(ranking.columns)]
ra_order = list(this_table_target_list.sort_values("ra")["Target Name"])
extra_cols = ["RA", "Dec", "Teff"]
for extra_col in extra_cols:
    ranking.loc[extra_col] = [target_list[target_list["Target Name"] == col][extra_col].values[0] for col in ra_order]
new_ordering = [*ranking.index[-len(extra_cols) :], *ranking.index[: -len(extra_cols)]]
final = ranking.reindex(new_ordering)
final.to_csv("final.csv")
final

,TIC 283940788,TIC 307119043,TIC 389836747,TIC 328181241,TIC 454140642,TIC 336882813,TIC 78568780,TIC 123098844,TIC 322727163,TIC 25818450,TIC 470710327
RA,00:35:24.34,00:59:18.61,01:33:10.91,02:52:36.95,04:19:05.63,06:09:33.75,06:51:23.75,18:38:17.48,20:38:51.99,23:30:58.43,23:49:18.97
Dec,+62:54:05.74,+51:13:17.91,+61:35:07.11,+03:20:52.38,+00:54:00.15,+14:37:44.35,-22:10:01.94,+44:41:54.96,+50:28:00.55,+53:04:08.94,+61:57:46.03
Teff,7384.0,7815.0,8874.0,5183.0,6592.0,6121.22,7199.0,6761.0,7876.56,7172.0,8986.0
2024-11-23 01:00,* *,* *,* *,,,,,* * *,* * *,* *,* *
2024-11-23 02:00,* * *,* *,* *,,,,,* * *,* * *,* *,* *
2024-11-23 03:00,* * *,* * *,* *,* *,,,,,* *,* *,* *
2024-11-23 04:00,* * *,* * *,* *,* *,,,,,* *,* *,* *
2024-11-23 05:00,* *,* * *,* *,* *,* * *,,,,,* *,* *
2024-11-23 06:00,* *,* * *,* *,* *,* * *,* *,,,,* *,* *
2024-11-23 07:00,* *,* *,* *,* *,* * *,* *,,,,* *,* *


In [87]:
tl.target_list[tl.target_list["Target Type"] == "RV Standard"].sort_values("Teff")[["Target Name", "Teff", "RA", "Dec"]]

,Target Name,Teff,RA,Dec
0,HD 18884,2000.0,03:02:16.77,04:05:23.10
1,HD 92588,3600.0,10:41:24.19,-01:44:29.40
2,HD 3765,3800.0,00:40:49.27,40:11:13.80
3,HD 23169,5200.0,03:43:53.14,25:43:31.50
4,HD 65583,5800.0,08:00:32.13,29:12:44.50
5,HD 693,6750.0,00:11:15.86,-15:28:04.70
6,BD +28 3402,6900.0,19:35:00.36,29:05:14.10
7,HD 112299,7100.0,12:55:28.29,25:44:17.00
8,HD 102870,7350.0,11:50:41.72,01:45:53.00


In [92]:
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list)
print(readme)

Project ID: UV-2024B-007
     Title: Characterizing Quadruple and Sextuple Eclipsing Binaries with PEPSI

Instrument: PEPSI PFU

PI: Steven Majewski

Run Contact:
   Name: Steven Majewski (UVA) / Robert Wilson (NASA)
   Phone: srm4n@virginia.edu   / 210.487.0187
   Contact Times: all / all

Conditions Required:
   Moon: Any
   Desired Seeing: Any
   Cloud Cover: Any

Program Summary:

Targets are TESS identified eclipsing quadruple systems (2 sets of Eclipsing Binaries, each showing both a Primary and Secondary Eclipse).

Our goal for this observing session is to obtain unambiguous radial velocity measurements for these systems.
We therefore seek observations when one of the two binary systems is in eclipse and the other is halfway between eclipses.

 Target Name       RA         Dec       Vmag    Teff  Fiber BLUE Cross Disperser BLUE CD NExp BLUE CD Exp Time RED Cross Disperser RED CD NExp RED CD Exp Time Desired BLUE SNR Desired RED SNR Priority       Notes
------------- ----------- 